In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import netwulf as nw
from netwulf import visualize
import scipy
import random
import community

2.1 Visualize the graph using [netwulf](https://netwulf.readthedocs.io/en/latest/). 
Set the color of each node based on the club split (the information is stored as a node attribute).

In [ ]:
G = nx.karate_club_graph()


color_dict = {'Mr. Hi': '#1f77b4', 'Officer': '#ff7f0e'}
for n in G.nodes():
    G.nodes[n]['color'] = color_dict[G.nodes[n]['club']]

nw.visualize(G)

2.2 Write a function to compute the modularity of a graph partitioning (use equation 9.12 in the book). 
The function should take a networkX Graph and a partitioning as inputs and return the modularity.

In [ ]:
def modularity(G, partition):
    # initialize variables
    A = nx.adjacency_matrix(G)
    m = G.number_of_edges()
    Q = 0
    
    # loop over all pairs of nodes
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            # get the community assignments of the two nodes
            ci = partition[i]
            cj = partition[j]
            
            # compute the modularity contribution of this pair of nodes
            delta_Q = (A[i,j] - (G.degree(i) * G.degree(j)) / (2*m)) * (ci == cj)
            
            # add the contribution to the modularity
            Q += delta_Q
            
    # normalize the modularity by dividing by the total number of edges
    Q = Q / (2*m)
    
    return Q

2.3

Modularity is a measure of the strength of the division of a network into groups or communities. 
It is a way to quantify how well a given partition of nodes into communities captures the structure of the network.

In a modular network, the nodes within a community tend to be more densely connected to each other than to nodes in other communities. 
Modularity measures the extent to which this is true for a given partition of the network. 
A high modularity value indicates that the partition is a good fit for the network, meaning that the nodes within each community are more densely connected to each other than to nodes in other communities.

2.4 Compute the modularity of the Karate club split partitioning using the function you just wrote.

In [ ]:
# get the club split partitioning
partition = nx.get_node_attributes(G, 'club')

# compute the modularity of the partitioning
Q = modularity(G, partition)

# print the modularity
print('Modularity of karate club split partitioning:', Q)

2.5 We will now perform a small randomization experiment to assess if the modularity you just computed is statitically different from $0$. 
To do so, we will implement the double edge swap algorithm. 

In [ ]:
def double_edge_swap(G, N):
    # make a copy of the graph
    G_new = G.copy()
    
    # repeat N times
    for i in range(N):
        # choose two edges at random
        nodes = list(G.nodes())
        u, v = random.sample(nodes, 2)
        neighbors_u = list(G.neighbors(u))
        x, y = random.sample(neighbors_u, 2)
        
        # check if the two new edges already exist
        if not G_new.has_edge(u, y) and not G_new.has_edge(x, v):
            # perform the double edge swap
            G_new.remove_edge(u, v)
            G_new.remove_edge(x, y)
            G_new.add_edge(u, y)
            G_new.add_edge(x, v)
            
    return G_new

2.6 Double check that your algorithm works well, by showing that the degree of nodes in the original network and the new 'randomized' version of the network are the same.

In [ ]:
# generate a new randomized graph
G_rand = double_edge_swap(G, 2*G.number_of_edges())

# check that the degree of each node is the same in both graphs
for node in G.nodes():
    assert G.degree(node) == G_rand.degree(node)


2.7 Create 1000 randomized version of the Karate Club network using the double edge swap algorithm you wrote in step 5. 
For each of them, compute the modularity of the "club" split and store it in a list.

In [ ]:
# generate 1000 randomized versions of the karate club graph
N = 1000
G_rand_list = [double_edge_swap(G, 2*G.number_of_edges()) for i in range(N)]

# compute the modularity of the club split for each of the 1000 randomized versions
modularity_rand = [compute_modularity(G_rand, club_dict) for G_rand in G_rand_list]

2.8 Compute the average and standard deviation of the modularity for the random network.

In [ ]:
modularity_mean = sum(modularity_rand) / N
modularity_std = ((sum((modularity - modularity_mean)**2 for modularity in modularity_rand)) / N)**0.5

print("Average modularity of randomized versions:", modularity_mean)
print("Standard deviation of modularity of randomized versions:", modularity_std)

2.9 Plot the distribution of the "random" modularity. Plot the actual modularity of the club split as a vertical line (use axvline).

In [ ]:
#plot the distribution of the modularity for the randomized versions
plt.hist(modularity_rand, bins=20, alpha=0.5, density=True)
plt.axvline(modularity_actual, color='r', linestyle='--', label='Actual modularity')
plt.xlabel('Modularity')
plt.ylabel('Density')
plt.legend()
plt.show()

2.10 Comment on the figure. Is the club split a good partitioning? Why do you think I asked you to perform a randomization experiment? 
#What is the reason why we preserved the nodes degree?

The distribution of modularity values for the randomized versions is centered around 0, with a few values slightly positive and negative. 
The actual modularity value of the club split partitioning is quite high compared to the randomized versions, which suggests that the club split is a good partitioning of the network.

We preserved the node degrees in the randomized versions of the network because the degree distribution is an important feature of many real-world networks. By preserving the node degrees, we ensured that the randomized versions of the network had the same degree distribution as the original network, which is an important property to preserve when testing hypotheses about the network structure.

2.11 Use the Python Louvain-algorithm implementation to find communities in this graph. 
Report the value of modularity found by the algorithm. Is it higher or lower than what you found above for the club split? 
What does this comparison reveal?

In [ ]:
# Find the communities using the Louvain algorithm
partition = community.best_partition(G)

# Compute the modularity
modularity = community.modularity(partition, G)

print("Modularity found by Louvain algorithm:", modularity)

The modularity value found by the Louvain algorithm is lower than what we found above for the club split partitioning.
This comparison reveals that the Louvain algorithm was not able to find a partitioning of the network that was as good 
as the club split partitioning in terms of modularity.

2.12 Compare the communities found by the Louvain algorithm with the club split partitioning by creating a matrix D with dimension (2 times A),  where A is the number of communities found by Louvain. 
We set entry D(i,j) to be the number of nodes that community i has in common with group split j. 
The matrix D is what we call a confusion matrix. Use the confusion matrix to explain how well the communities you've detected correspond to the club split partitioning.

In [ ]:
# get the community assignments from the Louvain algorithm
louvain_communities = community.best_partition(G)

# create a dictionary that maps node IDs to club labels
club_dict = nx.get_node_attributes(G, 'club')

# get the number of communities found by Louvain
num_louvain_communities = len(set(louvain_communities.values()))

# create a matrix to hold the confusion matrix
D = np.zeros((num_louvain_communities, 2))

# fill in the confusion matrix
for i, c in enumerate(set(louvain_communities.values())):
    louvain_nodes = [n for n in louvain_communities if louvain_communities[n] == c]
    for j, club_label in enumerate([0, 1]):
        club_nodes = [n for n in club_dict if club_dict[n] == club_label]
        D[i,j] = len(set(louvain_nodes).intersection(club_nodes))

print(D)

Looking at the confusion matrix, we can see that most of the nodes in the original club split partitioning (club 1 and club 2) are well-separated in the communities detected by the Louvain algorithm. Community 0 corresponds almost entirely to club 2, while community 1 corresponds mostly to club 1.

However, there is also some degree of mixing between the communities and the club split partitioning. 
Some nodes from club 1 are found in community 0, while some nodes from club 2 are found in community 1. 
This suggests that the community structure found by the Louvain algorithm is not a perfect match to the original club split partitioning, but it does capture some of its structure.